In [33]:
import torch
import numpy as np
import multiprocessing as mp

Computing efficiency violation

In [39]:
class compute_ev:
    def __init__(self, num_goods, P, preferences):
        """
        P: batch_size x n x n の二重確率行列 (torch.Tensor)
        preferences: n x n の選好行列 (torch.Tensor)
                     各行 i はエージェント i の選好を表し、値が大きいほど好む
        """
        self.P = P.clone().detach().cpu()
        if isinstance(preferences, np.ndarray):
            preferences = torch.tensor(preferences, dtype=torch.float32)
        self.preferences = preferences.clone().detach().float()
        self.n = num_goods

    def build_graph(self, Q, preferences):
        """
        Q: 現在の n x n 二重確率行列 (torch.Tensor)
        オブジェクトを頂点とするグラフを構築する。
        エッジ (a -> b) は、あるエージェント i が a を b より好む（preferences[i, a] > preferences[i, b]）
        かつ Q[i, b] > 0 である場合に追加する。
        エッジには (b, i, Q[i, b]) の情報を記録する。
        """

        graph = {a: [] for a in range(self.n)}
        for a in range(self.n):
            for b in range(self.n):
                if a == b:
                    continue
                for i in range(self.n):
                    if preferences[i, a].item() > preferences[i, b].item() and Q[i, b].item() > 0:
                        graph[a].append((b, i, Q[i, b].item()))
                        # 同じ (a, b) ペアについて、最初に条件を満たしたエージェントを witness とする
                        break
        return graph

    def find_cycle(self, graph):
        """
        DFS を用いてグラフ中のサイクル（閉路）を探索する。
        サイクルが見つかった場合は、サイクルを構成するエッジのリストを返す。
        各エッジは (from_object, to_object, witness_agent, available_probability) のタプル。
        サイクルがなければ None を返す。
        """
        visited = set()
        rec_stack = []  # 各要素は (vertex, edge_info)。最初の頂点は edge_info=None

        def dfs(v):
            visited.add(v)
            rec_stack.append((v, None))
            for (nbr, agent, avail) in graph[v]:
                for idx, (node, _) in enumerate(rec_stack):
                    if node == nbr:
                        cycle_edges = []
                        # rec_stack[idx+1:] に記録されているエッジ情報がサイクル内のエッジ
                        for j in range(idx + 1, len(rec_stack)):
                            edge = rec_stack[j][1]
                            if edge is not None:
                                cycle_edges.append(edge)
                        cycle_edges.append((v, nbr, agent, avail))
                        return cycle_edges
                if nbr not in visited:
                    rec_stack.append((v, (v, nbr, agent, avail)))
                    result = dfs(nbr)
                    if result is not None:
                        return result
                    rec_stack.pop()
            rec_stack.pop()
            return None

        for vertex in range(self.n):
            if vertex not in visited:
                cycle = dfs(vertex)
                if cycle is not None:
                    return cycle
        return None

    def divide_P_matrices(self, idx):
        Q = self.P[idx].clone()
        return Q
    
    def divide_preferences_matrices(self, idx):
        preferences = self.preferences[idx].clone()
        return preferences

    def execute_all_cycles(self, Q, preferences):
        """
        idx: バッチ内のインデックス
        対応する n x n 行列に対してサイクル交換を実施し、違反量 (violation) を計算する。
        """
        # 抽出: バッチ内の idx 番目の n x n 行列
        cycles_exchanges = []
        violation = 0.0
        while True:
            graph = self.build_graph(Q, preferences)
            cycle = self.find_cycle(graph)
            if cycle is None:
                break
            # サイクル内の各エッジの利用可能な確率の最小値を epsilon とする
            epsilons = [edge[3] for edge in cycle]
            epsilon = min(epsilons)
            violation += epsilon
            # サイクル内の各エッジについて交換を実施
            for (a, b, agent, avail) in cycle:
                Q[agent, b] -= epsilon
                Q[agent, a] += epsilon
            cycles_exchanges.append((cycle, epsilon))
        return violation

    def execute_all_cycles_batch(self):
        """
        バッチ内の各 n x n 行列に対して execute_all_cycles を計算し、違反量 (violation) をまとめる。
        結果はバッチサイズ x 1 のテンソルとなる。
        """
        batch_size = self.P.shape[0]

        # P行列とpreferences行列を分割
        P_list = [self.divide_P_matrices(i) for i in range(batch_size)]
        preferences_list = [self.divide_preferences_matrices(i) for i in range(batch_size)]

        with mp.Pool(mp.cpu_count()) as pool:
            # 各プロセスに対してバッチ内の異なる idx を渡して並列実行
            violations = pool.starmap(self.execute_all_cycles, zip(P_list, preferences_list))
        # 結果を torch.Tensor に変換し、形状を (batch_size, 1) にする
        violations = torch.tensor(violations, dtype=torch.float32).unsqueeze(1)
        return violations

Creating appropriate examples

In [35]:
def sinkhorn_knopp(matrix, max_iter=1000, tol=1e-6):
    for _ in range(max_iter):
        matrix /= matrix.sum(axis=1, keepdims=True)  # 行ごとに正規化
        matrix /= matrix.sum(axis=0, keepdims=True)  # 列ごとに正規化
        
        if np.allclose(matrix.sum(axis=1), 1, atol=tol) and np.allclose(matrix.sum(axis=0), 1, atol=tol):
            break
    return matrix

def generate_permutation_array(N, num_goods):
    P = np.zeros((N, num_goods))
    for i in range(N): P[i] = np.random.permutation(num_goods)
    return P

def sample_ranking(N, num_goods):
        """ 
        Samples ranked lists
        Arguments
            N: Number of samples
            prob: Probability of truncation       
        Returns:
            Ranked List of shape [N, Num_agents]
        """
        P = generate_permutation_array(N, num_goods) + 1
        
        return P/num_goods

def generate_batch(batch_size, num_goods):
        """
        Samples a batch of data from training
        Arguments
            batch_size: number of samples
            prob: probability of truncation
        Returns
            P: Agent's preferences, 
                P_{ij}: How much Agent-i prefers to be Good-j
        """
        N = batch_size * num_goods
        
        P = sample_ranking(N, num_goods)
        
        P = P.reshape(-1, num_goods, num_goods)                           
                
        return P


Test

In [40]:
np.random.seed(0)
batch_size = 2
num_goods = 4
randomassignment = np.random.rand(batch_size, 4, 4)
RandomAssignment = np.array([sinkhorn_knopp(matrix) for matrix in randomassignment])
preferences = generate_batch(batch_size, num_goods)
P = torch.tensor(RandomAssignment, dtype=torch.float32)
Preferences = torch.tensor(preferences, dtype=torch.float32)
print("P: ", P)
print("Preferences: ", Preferences)
ev= compute_ev(num_goods, P, Preferences)
violations = ev.execute_all_cycles_batch()
print("P: ", P)
print("Preferences: ", Preferences)
print(violations)

P:  tensor([[[0.1942, 0.2139, 0.3251, 0.2668],
         [0.1476, 0.1902, 0.2323, 0.4299],
         [0.2986, 0.1005, 0.3740, 0.2269],
         [0.3596, 0.4954, 0.0686, 0.0764]],

        [[0.0133, 0.2868, 0.4729, 0.2270],
         [0.4589, 0.1961, 0.1998, 0.1451],
         [0.1232, 0.3489, 0.1379, 0.3901],
         [0.4045, 0.1682, 0.1894, 0.2379]]])
Preferences:  tensor([[[0.5000, 0.7500, 0.2500, 1.0000],
         [0.5000, 0.7500, 1.0000, 0.2500],
         [0.7500, 0.2500, 1.0000, 0.5000],
         [0.7500, 0.5000, 1.0000, 0.2500]],

        [[0.7500, 0.5000, 0.2500, 1.0000],
         [0.2500, 0.5000, 1.0000, 0.7500],
         [1.0000, 0.7500, 0.5000, 0.2500],
         [0.2500, 0.7500, 0.5000, 1.0000]]])


RuntimeError: Broken pipe